## Прогнозирование на несколько шагов вперёд ансамблями над регрессионными деревьями

Регрессионные деревья и ансамбли над ними по умолчанию не предназначены для учёта временной структуры. Однако прогнозирование на несколько разных горизонтов одной и той же моделью, базирующейся на деревьях, можно осуществить следующим образом.

Возьмём обучающую выборку для задачи прогнозирования на один шаг вперёд. При группировке по идентификатору объекта применим операцию сдвига целевой переменной на $k$ шагов, где $k$ принимает значения от 0 до $(h-1)$, а $h$ — желаемый горизонт прогнозирования. Получим выборку, которая в $h$ раз больше по размеру, чем исходная. Добавим в эту выборку признак, принимающий значения от 1 до $h$ и равный $(k+1)$. Этот признак можно интерпретировать как то, на сколько шагов вперёд строится прогноз. Если теперь обучить модель на таких данных, то в зависимости от поданного на стадии предсказания значения нового признака модель сможет предсказывать на любое количество шагов вперёд от 1 до $h$.

## Многозадачное обучение (multi-task learning)

Допустим, есть вспомогательная задача, для которой имеется много размеченных данных, и есть основная задача, для которой размеченных данных меньше. Также допустим, что эти две задачи работают с одними и теми же объектами, но предсказывают на них разные целевые функции. Например, во вспомогательной задаче нужно определить пол человека с портретного фото, а в основной задаче требуется определить цвет волос.

В описанной ситуации можно поступить следующим образом. Построим нейронную сеть для решения вспомогательной задачи и обучим её. Затем уберём сколько-то последних слоёв получившейся сети, и вместо них поставим новые, такие что размерность выходного слоя уже соответствует основной задаче.

Далее есть два варианта:

1) Зафиксировать веса оставленных слоёв нейронной сети, обученной под вспомогательную задачу, и просто прогонять объекты через оставленные слои как через статичный предобработчик;

2) Веса оставшихся от вспомогательной задачи слоёв обучать наравне с весами новых слоёв, то есть считать, что они просто были инициализированы по-другому, но больше отличий между ними и весами новых слоёв нет.

Новая нейронная сеть обучится лучше, чем если бы сеть с такой же архитектурой обучалась на данных для основной задачи с нуля. Дело в том, что при описанном подходе нейронная сеть будет опираться на скрытое представление, выученное по большому количеству размеченных данных оставленными слоями старой нейронной сети. Также подстройка весов под разные задачи способствует увеличению обобщающей способности. Подробности о том, почему многозадачное обучение работает, есть в разделе 3 статьи [Ruder, 2017](https://arxiv.org/pdf/1706.05098.pdf).

Ещё один подход к многозадачному обучению таков: обучать нейронную сеть параллельно то под вспомогательную задачу, то под основную, при каждом переключении задачи оставляя сколько-то первых слоёв, но заменяя последние слои на то, что было ранее в задаче, к которой происходит переход.

## Как подобрать сложность модели без кросс-валидации?

Иногда в моделях из одного и того же семейства есть параметр, регулирующий сложность. Например, в семействе полиномиальных функций сложностью можно считать максимальную допустимую степень многочлена. Довольно часто сложность можно положить равной количеству оцениваемых по данным параметров — эта формулировка, в частности, включает в себя предыдущую, ведь при оценивании многочленом степени не выше $d$ оценивается $d+1$ параметр.

Модели из одного и того же семейства, отличающиеся друг от друга только сложностью, можно оценивать при помощи информационных критериев.

Информационный критерий Акаике имеет вид:
$$\mathrm{AIC} = 2k - 2l,$$
где $k$ — количество оценённых по данным параметров, а $l$ — значение функционала качества, достигнутое на тренировочной выборке (например, логарифм правдоподобия данных при условии того, что выборка порождается из распределения, задаваемого той моделью, которая была построена на тренировочной выборке).

Байесовский информационный критерий имеет вид:
$$\mathrm{BIC} = \log{(n)} \, k - 2l,$$
где $n$ — размер тренировочной выборки.

По сути, информационные критерии представляют собой оценку качества подгонки под данные с дополнительным штрафом за сложность модели. Интуитивно говоря, чем сложнее модель, тем больше ситуаций, под которые её можно подогнать и, значит, тем менее ценны уровни качества подгонки под данные.

Чем ниже значение какого-либо информационного критерия, тем более хорошей считается модель.

## Откуда может взяться недовольство заказчика машинно-обученной моделью.

Любая прикладная задача моделирования имеет заказчика, неважно, внутреннего или внешнего. Когда заказчик принимает модель, он решает, устраивает ли она его. Причины недовольства заказчика бывают такие:

1) Модель, и в самом деле, плохая;

2) Заказчик не умеет пользоваться моделью;

3) Ожидания заказчика неоправданно завышены.

Какой бы ни была истинная причина, заказчик, скорее всего, решит, что реализовался первый сценарий: модель плохая. Поэтому в интересах специалиста по моделированию избежать ситуаций, когда хорошая модель попадает во второй или третий сценарий.

Разберём эти два сценария подробнее.

Пример сценария, когда моделью не умеют пользоваться, таков: сервис на базе модели рассылает своевременные уведомления о появлении объектов положительного класса, но люди, которым они приходят, игнорируют их. Говоря более общо, можно сказать, что возможны осложнения с интеграцией в бизнес-процессы заказчика. Чтобы упредить их возникновение, полезно задать следующие вопросы:
* Что должно происходить с результатами работы сервиса?
* Как именно сервис приносит конечную ценность?
* Все ли исполнители знают и понимают ответы на два предыдущих вопроса?
* Можно ли оперативно отслеживать сбои в исполнении действий на базе сервиса и можно ли предложить количественные метрики оценки качества исполнения таких действий?
* Все ли признаки, использованные при обучении модели, будут доступны на стадии применения модели вовремя и в сопоставимом качестве?

Проблему завышенных ожиданий проще всего решить, ещё до старта работ по проекту согласовав метрику успеха и её целевой уровень. Сделать это можно, проведя оценку конечного эффекта от внедрения сервиса. Этот конечный эффект может быть как финансовым, то есть выражающимся в деньгах, так и каким-либо более широким (скажем, социальным), но, так или иначе, измеримым.

Впрочем, бывают ситуации, когда заказчик рассуждает не в терминах конечного эффекта, а в терминах всестороннего описания зависимостей — в этом случае заказчик может потребовать, скажем, почти стопроцентную точность по положительному классу. Разумеется, модель, не способная точно описать все зависимости, тоже может быть полезна. Проиллюстрировать это заказчику можно при помощи такого сравнения: если есть лотерея, где вероятность выиграть равна $p$, стоимость участия составляет $c$, а выигрыш приносит $g$, то ответ на вопрос, рационально ли участвовать в такой лотерее, зависит не от того, насколько вероятность $p$ близка к 100%, а от того, как друг с другом соотносятся все три указанных параметра, ведь ожидаемый чистый выигрыш равен $pg - c$.

## Субдискретизация (pooling)

Субдискретизация (pooling) устроена так: по входу слоя проходит фильтр (filter, kernel) и каждый раз от всего, что попадает в него, берёт значение некоторой агрегирующей функции, становящееся значением в соответствующем узле (нейроне) (unit, neuron). Например, в качестве агрегирующей функции можно взять максимум — тогда речь пойдёт о субдискретизации максимумом (max pooling).

Какие желаемые свойства добавляет субдискретизация максимумом?
* Возникает устойчивость к сдвигам изображения на небольшое количество пикселей;
* Появляется возможность реализовать следующую логику: присутствие детектируемого объекта на объединении участков зависит от того, присутствует ли объект хотя бы на одном из этих участков, а не от доли участков, на которых он присутствует.